In [1]:
import ollama

# Pull and load the model (e.g., Mistral or Llama2)
model_name = "deepseek-r1:1.5b"  # Change to the desired model (e.g., "llama2", "gemma")

# Query the model
response = ollama.chat(model=model_name, messages=[{"role": "user", "content": "meaning of name shevarthana?"}])

# Print response
print(response["message"]["content"])

ResponseError: timed out waiting for llama runner to start: 

In [ ]:
import ollama
import pandas as pd
import numpy as np

# Step 1: Preprocess ECG Data
def preprocess_ecg_data(csv_file):
    # Load the CSV file
    data = pd.read_csv(csv_file, header=None)
    time = data.iloc[:, 0].values
    ecg_signal = data.iloc[:, 1].values
    
    # Simulate peak detection (replace with actual peak detection logic)
    def detect_r_peaks(signal):
        # Example: Simple threshold-based peak detection
        threshold = np.mean(signal) + 0.5 * np.std(signal)
        peaks = np.where(signal > threshold)[0]
        return peaks
    
    r_peaks = detect_r_peaks(ecg_signal)
    
    # Extract features
    features = {
        'highest_point': np.max(ecg_signal),
        'lowest_point': np.min(ecg_signal),
        'avg_r_points': np.mean(ecg_signal[r_peaks]),
        'rr_interval': np.mean(np.diff(time[r_peaks])) if len(r_peaks) > 1 else 0,
        'qrs_duration': 0.08,  # Example value (replace with actual calculation)
        'qt_duration': 0.36,  # Example value (replace with actual calculation)
    }
    
    return features

# Step 2: Generate Summary Using Ollama
def generate_summary(features):
    # Prepare the prompt for the model
    prompt = f"""
    Analyze the following ECG data:
    - Highest Point: {features['highest_point']}
    - Lowest Point: {features['lowest_point']}
    - Average R-Points: {features['avg_r_points']}
    - RR Interval: {features['rr_interval']}
    - QRS Duration: {features['qrs_duration']}
    - QT Duration: {features['qt_duration']}

    Summarize the ECG data and provide insights into possible cardiac conditions.
    """
    
    # Query the Ollama model
    response = ollama.chat(
        model="medllama2",  # Use the medical-focused model
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response["message"]["content"]

# Step 3: Handle User Queries
def handle_user_query(query, features):
    # Prepare the prompt for the model
    prompt = f"""
    ECG Data Features:
    - Highest Point: {features['highest_point']}
    - Lowest Point: {features['lowest_point']}
    - Average R-Points: {features['avg_r_points']}
    - RR Interval: {features['rr_interval']}
    - QRS Duration: {features['qrs_duration']}
    - QT Duration: {features['qt_duration']}

    User Query: {query}

    Provide a detailed response to the user's query based on the ECG data.
    """
    
    # Query the Ollama model
    response = ollama.chat(
        model="medllama2",  # Use the medical-focused model
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response["message"]["content"]

# Step 4: Main Workflow
def main(csv_file, user_query=None):
    # Preprocess ECG data
    features = preprocess_ecg_data(csv_file)
    
    # Generate summary
    summary = generate_summary(features)
    print("ECG Summary:\n", summary)
    
    # Handle user query if provided
    if user_query:
        response = handle_user_query(user_query, features)
        print("\nUser Query Response:\n", response)

# Example Usage
csv_file = "ecg_data.csv"  # Replace with your CSV file path
user_query = "Are there any signs of blockage in the ECG data?"
main(csv_file, user_query)